## Libraries & Product id info

In [ ]:
# 한국어 설정
# 런타임 재시작 필요
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
# libraries
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

plt.rc('font', family='NanumBarunGothic')

%matplotlib inline

In [2]:
# functions

def review_crawler(product_code: str, page_num: int):
    final_review_list = []
    for num in tqdm(range(1, page_num+1)):
        url = 'https://review9.cre.ma/easydew.co.kr/products/reviews?product_code={}&app=0&iframe=1&iframe_id=crema-product-reviews-2&page={}'.format(product_code, str(num))
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            reviews = soup.select('div.review_list_v2__review_rcontent')
        else :
            print('status code --> ', response.status_code)

        review_list = []
        for review in reviews:
            reviewer_name = review.select_one('div.review_list_v2__user_name_message')
            review_types = review.select('span.review_options_v2__name')
            review_values = review.select('span.review_options_v2__value')
            review_dict = {}
            for review_type, review_val in zip(review_types, review_values):
                review_dict['reviewer'] = reviewer_name.text.split('님')[0].strip()
                review_dict[review_type.text.strip()] = review_val.text.strip()

            if review_dict != {}: # review_dict가 비어있지 않을 때만 리스트에 넣어줌 (비어있는 경우가 있기 때문에)
                review_list.append(review_dict)

        final_review_list += review_list

    review_df = pd.DataFrame(final_review_list)
    # review_df = review_df.iloc[:, :-1]
    return review_df

## Product id

In [ ]:
# products id

# 새로운 상품과 할인 상품을 통해 유입을 더 높이기? + 매출 증대에 영향

# -- 미백/기미 --
# EGF 기미엠플: 2073 (one of the products with a number of reviews)
# 기미크림: 2226 (one of the products with a number of reviews)
# 이지듀 EX DW-EGF 멜라토닝 패치 8매 (기미앰플 1ml 증정): 2079 (one of the products with a number of reviews)
# [~66%] 기미 토닝 2700샷: 2221

# -- 트러블/민감/진정 --
# [NEW 손상완화크림] 베리어 리페어 크림: 2227
# [런칭특가 50%] EGF 흔적세럼: 2303
# [런칭특가 50%] 아크네 클리어 SOS 스팟: 2288
# [런칭특가 ~62% ] EGF 필링세럼 + 해면: 2314 --> 페이지 20개

# -- 모공/탄력/주름 --
# DW-EGF 크림 프레좀 50ml: 1646
# 이지듀 DW-EGF 더마 톡스 니들샷: 2185

In [ ]:
# Cohort dashBoard template

# -- crawling --
review_df = review_crawler() # need to fill out
# removing useless columns
review_df = review_df.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df))

# preprocessing
review_df = review_df.dropna(subset=['연령대'])
review_df['연령대'] = review_df['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df.head(4)

# -- dashboard--
# Dashboard
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
age_list = list(sorted(review_df['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령별 이지듀 DW-EGF 더마 톡스 니들샷",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

## 미백/기미

### EGF 기미엠플

In [3]:
# -- crawling --
review_df = review_crawler('2073', 100) # need to fill out
# removing useless columns
review_df = review_df.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df))

# preprocessing
review_df = review_df.dropna(subset=['연령대'])
review_df['연령대'] = review_df['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

378


,reviewer,피부타입,연령대,피부고민,피부과 관리,멜라토닝 4SET,멜라토닝 앰플,필수옵션,추가구성상품(단독구매x),멜라토닝,...,1+1 특가 상품,[1+1] 기미 라인 특가,1. 기미 풀케어 라인,추가구성 상품,추가구성상품,추가구성,[BEST] EGF 기미앰플,기미앰플,시너지템 추가구성상품,2. 시너지템 추가구성상품
0,박****,중성,40대,기미/잡티,경험 있음,멜라토닝 앰플 1ml * 28,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,박****,복합성,50대,기미/잡티,경험 없음,NaN,대용량 30ml,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,이****,중성,40대,기미/잡티,경험 있음,NaN,NaN,[1+1] 대용량 30ml ■회원증정,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,정****,건성,40대,기미/잡티,경험 있음,NaN,[NEW] 대용량 (30ml) ■회원증정,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df['연령대'].unique())))

fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 EGF 기미엠플",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['20대', '30대', '40대', '50대']


### 기미크림

In [5]:
# -- crawling --
review_df_1 = review_crawler('2226', 100) # need to fill out
# removing useless columns
review_df_1 = review_df_1.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_1))

# preprocessing
review_df_1 = review_df_1.dropna(subset=['연령대'])
review_df_1['연령대'] = review_df_1['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_1.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

257


,reviewer,피부타입,연령대,피부고민,피부과 관리,[53%] 기미크림,기미크림,[56%] 기미크림
0,최****,중성,40대,홍조,경험 없음,NaN,NaN,NaN
1,장****,복합성,20대,트러블,경험 있음,NaN,NaN,NaN
2,유****,지성,50대,기미/잡티,경험 없음,[40%] 기미크림 1개 ■회원증정■ 기미패치 1매,NaN,NaN
3,박****,건성,50대,주름/영양,경험 있음,NaN,[58%] 기미크림 2개,NaN


In [12]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_1['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_1['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "10대"')['피부타입'],
    name='10대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "10대"')['피부고민'],
    name='10대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "10대"')['피부과 관리'],
    name='10대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_1.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 기미크림",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['10대', '20대', '30대', '40대', '50대']


### 이지듀 EX DW-EGF 멜라토닝 패치 8매 : 2079

In [7]:
# -- crawling --
review_df_2 = review_crawler('2079', 100) # need to fill out
# removing useless columns
review_df_2 = review_df_2.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_2))

# preprocessing
review_df_2 = review_df_2.dropna(subset=['연령대'])
review_df_2['연령대'] = review_df_2['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_2.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

225


,reviewer,피부타입,연령대,피부고민,피부과 관리,[67%] 기미패치,[단 하루 67%] 기미패치,[BEST] 더마니들 4개입
0,김****,복합성,50대,기미/잡티,경험 있음,NaN,NaN,NaN
1,김****,복합성,40대,화이트닝,NaN,NaN,NaN,NaN
2,박****,복합성,40대,기미/잡티,경험 있음,NaN,NaN,NaN
3,조****,복합성,40대,화이트닝,NaN,NaN,NaN,NaN


In [13]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_2['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_2['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_2.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['20대', '30대', '40대', '50대']


### 기미 토닝 2700샷: 2221

In [9]:
# -- crawling --
review_df_3 = review_crawler('2221', 100) # need to fill out
# removing useless columns
review_df_3 = review_df_3.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_3))

# preprocessing
review_df_3 = review_df_3.dropna(subset=['연령대'])
review_df_3['연령대'] = review_df_3['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_3.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

406


,reviewer,피부타입,연령대,피부고민,피부과 관리,기미 토닝 2700샷,추가구성,1개월 관리,[단 하루 66%],[BEST] 더마니들 4개입
0,윤****,지성,30대,트러블,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN
1,최****,건성,40대,기미/잡티,경험 있음,기미앰플(1ml) 4개입 + 더마니들 4개입 + 롤러,NaN,NaN,NaN,NaN
2,김****,복합성,40대,모공/피지,경험 있음,기미앰플(30ml) 대용량 + 더마니들 8개입 + 롤러,NaN,NaN,NaN,NaN
3,김****,건성,50대,기미/잡티,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN


In [14]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_3['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_3['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_3.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['20대', '30대', '40대', '50대']


## 트러블/민감/진정

### [NEW 손상완화크림] 베리어 리페어 크림: 2227

In [15]:
# -- crawling --
review_df_4 = review_crawler('2227', 100) # need to fill out
# removing useless columns
review_df_4 = review_df_4.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_4))

# preprocessing
review_df_4 = review_df_4.dropna(subset=['연령대'])
review_df_4['연령대'] = review_df_4['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_4.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

265


,reviewer,피부타입,연령대,피부고민,피부과 관리,[54%] 손상회복크림,모이스트 크림,[~50%] 손상회복크림,손상회복크림,멜라토닝 앰플
0,송****,건성,50대,기미/잡티,경험 있음,[50%] 손상회복크림 1개,NaN,NaN,NaN,NaN
1,박****,건성,40대,민감성,경험 있음,NaN,[39%] 크림 모이스트 3개,NaN,NaN,NaN
2,오****,복합성,20대,민감성,경험 없음,NaN,[20%] 크림 모이스트 1개,NaN,NaN,NaN
3,박****,복합성,20대,민감성,경험 없음,NaN,NaN,NaN,NaN,NaN


In [16]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_4['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_4['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "10대"')['피부타입'],
    name='10대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "10대"')['피부고민'],
    name='10대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "10대"')['피부과 관리'],
    name='10대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_4.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['10대', '20대', '30대', '40대', '50대']


### [런칭특가 50%] EGF 흔적세럼: 2303

In [17]:
# -- crawling --
review_df_5 = review_crawler('2303', 100) # need to fill out
# removing useless columns
review_df_5 = review_df_5.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_5))

# preprocessing
review_df_5 = review_df_5.dropna(subset=['연령대'])
review_df_5['연령대'] = review_df_5['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_5.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

338


,reviewer,피부타입,연령대,피부고민,피부과 관리,EGF 흔적세럼,트러블 라인,흔적세럼,추가 구성 상품
0,채엽,복합성,20대,트러블,경험 있음,NaN,NaN,NaN,NaN
1,김****,복합성,30대,트러블,경험 있음,NaN,NaN,NaN,NaN
2,장****,복합성,50대,트러블,경험 없음,[50%] 흔적라인 풀세트 [증정] 흔적미백패치 4매,NaN,NaN,NaN
3,박****,복합성,30대,트러블,경험 있음,[50%] 아크네 클리어 흔적세럼,NaN,NaN,NaN


In [18]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_5['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_5['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "10대"')['피부타입'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "10대"')['피부고민'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "10대"')['피부과 관리'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_5.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['10대', '20대', '30대', '40대', '50대']


### [런칭특가 50%] 아크네 클리어 SOS 스팟: 2288

In [19]:
# -- crawling --
review_df_6 = review_crawler('2288', 100) # need to fill out
# removing useless columns
review_df_6 = review_df_6.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_6))

# preprocessing
review_df_6 = review_df_6.dropna(subset=['연령대'])
review_df_6['연령대'] = review_df_6['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_6.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

423


,reviewer,피부타입,연령대,피부고민,피부과 관리,트러블 라인,추가 구성 상품
0,김****,복합성,40대,기미/잡티,경험 있음,NaN,NaN
1,채엽,복합성,20대,트러블,경험 있음,NaN,NaN
2,안****,복합성,50대,홍조,경험 있음,NaN,NaN
3,김****,복합성,30대,트러블,경험 있음,NaN,NaN


In [20]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_6['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_6['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "10대"')['피부타입'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "10대"')['피부고민'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "10대"')['피부과 관리'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_6.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['10대', '20대', '30대', '40대', '50대']


### [런칭특가 ~62% ] EGF 필링세럼 + 해면: 2314 --> 페이지 20개

In [21]:
# -- crawling --
review_df_7 = review_crawler('2314', 20) # need to fill out
# removing useless columns
review_df_7 = review_df_7.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_7))

# preprocessing
review_df_7 = review_df_7.dropna(subset=['연령대'])
review_df_7['연령대'] = review_df_7['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_7.head(4)

  0%|          | 0/20 [00:00<?, ?it/s]

83


,reviewer,피부타입,연령대,피부고민,피부과 관리,필링세럼,추가구성상품
0,윤****,지성,30대,트러블,경험 있음,[61%] 필링세럼 4개,NaN
1,유****,복합성,40대,트러블,경험 있음,[61%] 필링세럼 4개,NaN
2,현****,복합성,30대,트러블,경험 있음,[50%] 필링세럼 1개,NaN
3,장****,복합성,50대,트러블,경험 없음,NaN,[60%] 아크네 클리어 모공청소토너


In [22]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_7['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_7['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "10대"')['피부타입'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "10대"')['피부고민'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "10대"')['피부과 관리'],
    name='10대',
    marker_color='orange',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='indianred',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='crimson',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_7.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['10대', '20대', '30대', '40대', '50대']


## 모공/탄력/주름

### DW-EGF 크림 프레좀 50ml: 1646

In [23]:
# -- crawling --
review_df_8 = review_crawler('1646', 100) # need to fill out
# removing useless columns
review_df_8 = review_df_8.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_8))

# preprocessing
review_df_8 = review_df_8.dropna(subset=['연령대'])
review_df_8['연령대'] = review_df_8['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_8.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

123


,reviewer,피부타입,연령대,피부고민,프레좀,피부과 관리,[37%] 프레좀 탄력크림,[45%] 프레좀 탄력크림,프레좀 탄력크림
0,곽****,중성,30대,민감성,[20%] 크림 프레좀 2개,NaN,NaN,NaN,NaN
1,이****,건성,50대,주름/영양,[10%] 크림 프레좀 1개,경험 있음,NaN,NaN,NaN
2,강****,건성,40대,주름/영양,[10%] 크림 프레좀 1개,경험 없음,NaN,NaN,NaN
3,나****,복합성,20대,민감성,[10%] 크림 프레좀 1개,NaN,NaN,NaN,NaN


In [24]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_8['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_8['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_8.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['20대', '30대', '40대', '50대']


### 이지듀 DW-EGF 더마 톡스 니들샷: 2185

In [25]:
# -- crawling --
review_df_9 = review_crawler('2185', 100) # need to fill out
# removing useless columns
review_df_9 = review_df_9.iloc[:, :] # can be changed depending on the columns crawled
# checking length of data
print(len(review_df_9))

# preprocessing
review_df_9 = review_df_9.dropna(subset=['연령대'])
review_df_9['연령대'] = review_df_9['연령대'].apply(lambda x: x.split('대')[0] + '대')

# checking data
review_df_9.head(4)

  0%|          | 0/100 [00:00<?, ?it/s]

411


,reviewer,피부타입,연령대,피부고민,피부과 관리,기미 토닝 2700샷,더마니들(롤러미포함),추가구성,1개월 관리,[단 하루 66%]
0,윤****,지성,30대,트러블,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN
1,최****,건성,40대,기미/잡티,경험 있음,기미앰플(1ml) 4개입 + 더마니들 4개입 + 롤러,NaN,NaN,NaN,NaN
2,김****,복합성,40대,모공/피지,경험 있음,기미앰플(30ml) 대용량 + 더마니들 8개입 + 롤러,NaN,NaN,NaN,NaN
3,김****,건성,50대,기미/잡티,경험 있음,기미앰플(1ml) 2개입 + 더마니들 2개입 + 롤러,NaN,NaN,NaN,NaN


In [26]:
# -- dashboard--
# Dashboard
print(list(sorted(review_df_9['연령대'].unique())))
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('<피부타입>', '<피부고민>', '<피부과 관리>'),
                    shared_yaxes=True)

# spliting by age cohort
# age_list = list(sorted(review_df_9['연령대'].unique()))

# 피부타입
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "20대"')['피부타입'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.update_yaxes(title_text='리뷰수', row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "30대"')['피부타입'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "40대"')['피부타입'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "50대"')['피부타입'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2
), row=1, col=1)

# 피부고민
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "20대"')['피부고민'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "30대"')['피부고민'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "40대"')['피부고민'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "50대"')['피부고민'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=2)

# 피부과 관리
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "20대"')['피부과 관리'],
    name='20대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "30대"')['피부과 관리'],
    name='30대',
    marker_color='grey',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "40대"')['피부과 관리'],
    name='40대',
    marker_color='royalblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)
fig.add_trace(go.Histogram(
    histfunc='count',
    x=review_df_9.query('연령대 == "50대"')['피부과 관리'],
    name='50대',
    marker_color='lightblue',
    marker_line_color='black',
    marker_line_width=2,
    showlegend=False
), row=1, col=3)

fig.update_layout(height=450, width=1200,
                  title={'text': "연령대별 이지듀 EX DW-EGF 멜라토닝 패치 8매",
                        'y':0.95,
                        'x':0.49,
                        'xanchor': 'center',
                        'yanchor': 'top',
                         'font_size' : 25},
                  plot_bgcolor='white',
                  )

fig.show()

['20대', '30대', '40대', '50대']


## 광고 가격

    https://www.adjust.com/ko/glossary/roas-definition/
    https://www.hellodigital.kr/blog/dmkt-general-plain-guide-roas/

    ROAS: 광고비의 효율성

### 카카오톡 배너 광고

    https://m.blog.naver.com/connectree/222979155098
    https://store.cafe24.com/kr/story/2197
    

In [111]:
def get_roi_roas_from_kakao(conversion_rate, product_price):
    # 일에 30만원의 광고비, 한번 클릭당 1000원의 비용발생이라고 가정할 때
    click_num = 300000/1000
    buying_customer_num = click_num*conversion_rate
    daily_sales = buying_customer_num*product_price
    daily_profit = daily_sales - 300000

    monthly_sales = daily_sales*30
    monthly_profit = daily_profit*30
    print('월 매출: ', monthly_sales)
    print('월 순수익: ', monthly_profit)
    print()
    roas = monthly_sales/(300000*30)
    roi = monthly_profit/(300000*30)

    print('ROAS: ', roas)
    print('ROI: ', roi)
    return

In [112]:
get_roi_roas_from_kakao(0.04, 51000)

월 매출:  18360000.0
월 순수익:  9360000.0

ROAS:  2.04
ROI:  1.04


In [114]:
get_roi_roas_from_kakao(0.06, 51000) # 전환율이 4% 높아진다면?

월 매출:  27540000.0
월 순수익:  18540000.0

ROAS:  3.06
ROI:  2.06


### 카카오 푸쉬 메세지

    https://cs.kakao.com/helps?service=110&category=653&device=2467&locale=KOREAN&articleId=1073198347&controllerName=help&actionName=mobileviewpage&accountLoginUrl=https%3A%2F%2Faccounts.kakao.com%2F&without_layout=false#:~:text=%EB%A9%94%EC%8B%9C%EC%A7%80%20%EA%B4%91%EA%B3%A0%EB%8A%94%20%EB%B0%9C%EC%86%A1%EB%8B%B9,%EA%B1%B4%20%EB%8B%B9%2020%EC%9B%90%EC%9E%85%EB%8B%88%EB%8B%A4.
    

In [ ]:
# 건당 20원

### 네이버 밴드 피드 광고

    https://m.blog.naver.com/gskweon65/221964021571
    https://ssl.pstatic.net/cmstatic/ad_guide/native_catalog/ko/native_catalog_6.6.pdf?timestamp=1699262958011
    https://jpddung.tistory.com/entry/DA-%ED%83%80%EA%B2%9F%EC%9D%B4-40%EB%8C%80-%EC%9D%B4%EC%83%81%EC%9D%B4%EB%A9%B4-%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%B0%B4%EB%93%9C%EA%B0%80-%EC%A0%81%ED%95%A9%ED%95%9C-%EB%A7%A4%EC%B2%B4%EC%9D%BC%EA%B9%8C%EC%9A%94-NAVER-BAND-%EB%A7%A4%EC%B2%B4-%EB%B6%84%EC%84%9D

In [ ]:
# 타겟팅 가능
# cpc 800원 (노출보다는 효율을 보기 위해)
# 2~5% 전환율

### 메타 광고

#### 인스타그램 광고